<a href="https://colab.research.google.com/github/juanfranbrv/curso-langchain/blob/main/Cadenas%20b%C3%A1sicas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Nuevas cadenas con LangChain Expression Language (LCEL)
---

En octubre de 2023, LangChain introdujo una nueva forma de trabajar con cadenas, más eficiente y simple, utilizando el operador |.  

Esto permite encadenar elementos de manera intuitiva, con soporte para stream, batch y operaciones asíncronas.  

Además, con LCEL (LangChain Expression Language), las cadenas se pueden definir de forma declarativa y funcional, eliminando la necesidad de clases complejas.   

Aunque las cadenas "antiguas" siguen funcionando, es recomendable adoptar esta nueva sintaxis.

### Ventajas del nuevo sistema:

- **Alineación con el framework:** La nueva sintaxis es el estándar recomendado para futuros proyectos.
- **Interface unificada:** Simplifica la integración de múltiples métodos.
- **Facilidad de composición:** Permite estructurar cadenas de forma secuencial, paralela o con fallbacks.
- **Código más limpio:** Menos líneas para lograr los mismos resultados.

## Configuración del entorno del cuaderno
---

Configuramos el entorno de trabajo para utilizar LangChain con distintos modelos de lenguaje (LLMs).  
1. Obtenemos las claves API para acceder a los servicios de OpenAI, Groq, Google y Hugging Face.

2. Instalamos la librería LangChain y las integraciones necesarias para cada uno de estos proveedores.

3. Importamos las clases específicas de LangChain que permiten crear plantillas de prompts e interactuar con los diferentes modelos de lenguaje, dejándolo todo listo para empezar a desarrollar aplicaciones basadas en LLMs. (Este codigo se explico con detalle en el primer cuaderno)

Comenta (#) las librerias y modelos que no desees usar.

In [1]:
# Importar la librería `userdata` de Google Colab.
# Esta librería se utiliza para acceder a datos de usuario almacenados de forma segura en el entorno de Colab.
from google.colab import userdata

# Obtener las claves API de diferentes servicios desde el almacenamiento seguro de Colab.
OPENAI_API_KEY=userdata.get('OPENAI_API_KEY')
GROQ_API_KEY=userdata.get('GROQ_API_KEY')
GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')
HUGGINGFACEHUB_API_TOKEN=userdata.get('HUGGINGFACEHUB_API_TOKEN')

# Instalar las librerías necesarias usando pip.
# El flag `-qU` instala en modo silencioso (`-q`) y actualiza las librerías si ya están instaladas (`-U`).
!pip install langchain -qU  # Instalar la librería principal de LangChain.

# Instalar las integraciones de LangChain con diferentes proveedores de LLMs.
!pip install langchain-openai -qU
!pip install langchain-groq -qU
!pip install langchain-google-genai -qU
!pip install langchain-huggingface -qU

# Importar las clases necesarias de LangChain para crear plantillas de prompt.
# `ChatPromptTemplate` es la clase base para plantillas de chat.
# `SystemMessagePromptTemplate` se usa para mensajes del sistema (instrucciones iniciales).
# `HumanMessagePromptTemplate` se usa para mensajes del usuario.
from langchain.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate

# Importar las clases para interactuar con los diferentes LLMs a través de LangChain.
from langchain_openai import ChatOpenAI
from langchain_groq import ChatGroq
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_huggingface import HuggingFaceEndpoint

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.6/411.6 kB 21.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 454.3/454.3 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 39.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.1/109.1 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 1.5 MB/s eta 0:00:00


In [ ]:
# Configurar el modelo
llm1 = ChatGoogleGenerativeAI(
    model="gemini-pro",
    api_key=GOOGLE_API_KEY,
    temperature=0.7
)

llm2 = ChatOpenAI(model="gpt-4o-mini",
                 api_key=OPENAI_API_KEY,
                 temperature=0.7)

In [ ]:
# Definir las plantillas
template_sistema = """\
Eres un tutor amigable de {idioma}. Tu trabajo es:
1. Mantener una conversación simple con el estudiante
2. Corregir errores básicos de gramática y ortografía si los hay
3. Ser paciente y motivador
"""

template_usuario = """\
El estudiante dice: {mensaje_usuario}

Por favor, responde de manera natural y si hay algún error gramatical,
corrígelo sutilmente en tu respuesta.
"""

In [ ]:
# Crear el ChatPromptTemplate
chat_prompt = ChatPromptTemplate.from_messages([
    SystemMessagePromptTemplate.from_template(template_sistema),
    HumanMessagePromptTemplate.from_template(template_usuario),
])

# Podria ser asi ???? explicar

# Crear el ChatPromptTemplate
chat_prompt = ChatPromptTemplate.from_messages([
    ("system", template_sistema),
    ("human", template_usuario),
])


In [ ]:
def chatear_en_idioma(mensaje_usuario, idioma="español"):
    """
    Función simple para procesar mensajes del usuario y obtener respuestas
    1. Recibe el mensaje del usuario
    2. Lo inyecta en el chatprompttemplate
    3. Invoca el modelo con el
    4. Devuelve la respuesta
    """
    # Formatear el prompt con el mensaje del usuario y el idioma
    prompt = chat_prompt.format(
        mensaje_usuario=mensaje_usuario,
        idioma=idioma
    )

    # Obtener la respuesta del modelo
    respuesta = llm.invoke(prompt)

    return respuesta.content

In [ ]:
    print("¡Bienvenido al Chatbot de Aprendizaje de Idiomas!")
    print("Escribe 'salir' para terminar la conversación")

    while True:
        # Obtener entrada del usuario
        mensaje = input("\nTú: ")

        if mensaje.lower() == 'salir':
            print("¡Hasta luego! Gracias por practicar.")
            break

        # Obtener y mostrar la respuesta
        respuesta = chatear_en_idioma1(mensaje)
        print("\nTutor:", respuesta)

¡Bienvenido al Chatbot de Aprendizaje de Idiomas!
Escribe 'salir' para terminar la conversación

Tú: Ola

Tutor: ¡Hola! Me alegro de que estés aquí. ¿Cómo te llamas?

Tú: me a mordido un perro

Tutor: ¡Ay, lo siento mucho! ¿Estás bien? ¿Te duele mucho?


KeyboardInterrupt: Interrupted by user

Refs:

1. https://www.paradigmadigital.com/dev/uso-de-cadenas-langchain-gen-ai/

2. https://python.langchain.com/v0.1/docs/modules/chains/

3. https://medium.com/@anuragmishra_27746/practical-hands-on-with-langchain-expression-language-lcel-for-building-langchain-agent-chain-2a9364dc4ca3

4. https://medium.com/@itsmybestview/unbridling-the-power-of-langchain-framework-with-lcel-9e5f7bf8af74

5. https://www.youtube.com/playlist?list=PLGPnu4k-KSC1s7apArPZz1AavDJ4Mvitd

6. https://www.youtube.com/watch?v=LzxSY7197ns&t=12s

7. https://www.youtube.com/watch?v=H8DrR2pXbww&list=PLilZ1IiRt0R26etAQeQ9xa5h5UzK18zie&index=3

8. https://www.youtube.com/watch?v=8BV9TW490nQ&t=523s

9. https://www.youtube.com/watch?v=8BV9TW490nQ


Usar esto para localizar
https://www.youtube.com/results?search_query=langchain+lcel+
